In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, MegaForSequenceClassification
import torchinfo
torch.__version__

/Users/alan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'2.2.0'

In [2]:
device = torch.device("cpu")
device

device(type='cpu')

In [3]:
tokenizer = AutoTokenizer.from_pretrained("mnaylor/mega-base-wikitext")
model = MegaForSequenceClassification.from_pretrained("mnaylor/mega-base-wikitext")
model.to(device)

Some weights of MegaForSequenceClassification were not initialized from the model checkpoint at mnaylor/mega-base-wikitext and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MegaForSequenceClassification(
  (mega): MegaModel(
    (embedding_layer): MegaEmbeddings(
      (word_embeddings): Embedding(50265, 128, padding_idx=1)
    )
    (layers): ModuleList(
      (0-3): 4 x MegaBlock(
        (mega_layer): MegaMovingAverageGatedAttention(
          (activation): SiLU()
          (dropout): MegaDropout()
          (hidden_dropout): MegaDropout()
          (attention_dropout): MegaDropout()
          (norm): MegaSequenceNorm(
            (norm): MegaScaleNorm()
          )
          (ema_gate): MegaMultiDimensionDampedEma()
          (v_proj): Linear(in_features=128, out_features=256, bias=True)
          (mx_proj): Linear(in_features=128, out_features=576, bias=True)
          (h_proj): Linear(in_features=256, out_features=128, bias=True)
          (rel_pos_bias): MegaRotaryRelativePositionalBias()
          (softmax): Softmax(dim=-1)
        )
        (nffn): MegaNormalizedFeedForwardNetwork(
          (activation): SiLU()
          (dropout): MegaDropout()

In [4]:
torchinfo.summary(model)

Layer (type:depth-idx)                                            Param #
MegaForSequenceClassification                                     --
├─MegaModel: 1-1                                                  --
│    └─MegaEmbeddings: 2-1                                        --
│    │    └─Embedding: 3-1                                        6,433,920
│    └─ModuleList: 2-2                                            --
│    │    └─MegaBlock: 3-2                                        223,042
│    │    └─MegaBlock: 3-3                                        223,042
│    │    └─MegaBlock: 3-4                                        223,042
│    │    └─MegaBlock: 3-5                                        223,042
├─MegaClassificationHead: 1-2                                     --
│    └─Linear: 2-3                                                16,512
│    └─Dropout: 2-4                                               --
│    └─Linear: 2-5                                                2

In [5]:
tokenizer

RobertaTokenizerFast(name_or_path='mnaylor/mega-base-wikitext', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [6]:
data_check_test = load_dataset(
    "csv", data_files="data/tweets.csv"
    )
print(data_check_test['train'][0])

def preprocess_function(examples):
    # Add your binary labels based on the 'class' column
    examples['hate_speech'] = 1 if examples['class'] == 0 else 0
    examples['offensive_language'] = 1 if examples['class'] == 1 else 0
    examples['neither'] = 1 if examples['class'] == 2 else 0
    return examples

data_check_test = data_check_test.map(preprocess_function)
print(data_check_test['train'][0])
# Dataset only have training data, hence split it into train/test
data_check_test = data_check_test['train'].train_test_split(test_size=0.5)

{'idx': 0, 'count': 3, 'hate_speech': 0, 'offensive_language': 0, 'neither': 3, 'class': 2, 'tweet': "!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."}
{'idx': 0, 'count': 3, 'hate_speech': 0, 'offensive_language': 0, 'neither': 1, 'class': 2, 'tweet': "!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."}


In [7]:
def tokenize_func(examples):
    #  padding: 'max_length': pad to a length specified by the max_length argument or the
    #  maximum length accepted by the model if no max_length is provided (max_length=None).
    #  Padding will still be applied if you only provide a single sequence. [from documentation]


    #  truncation: True or 'longest_first': truncate to a maximum length specified
    #  by the max_length argument or the maximum length accepted by the model if
    #  no max_length is provided (max_length=None). This will truncate token by
    #  token, removing a token from the longest sequence in the pair until the
    #  proper length is reached. [from documentation]
    return tokenizer(
        examples["tweet"], padding="max_length", truncation=True
        )

In [8]:
tokenized_data = data_check_test.map(tokenize_func, batched=True)


Map:   0%|          | 0/12391 [00:00<?, ? examples/s]

Map:   0%|          | 0/12392 [00:00<?, ? examples/s]

In [9]:
print(tokenized_data)
tokenized_data = tokenized_data.remove_columns([
    "idx", "count", "class", "tweet"])
print(tokenized_data['train'][0])
tokenized_data.set_format("torch")

DatasetDict({
    train: Dataset({
        features: ['idx', 'count', 'hate_speech', 'offensive_language', 'neither', 'class', 'tweet', 'input_ids', 'attention_mask'],
        num_rows: 12391
    })
    test: Dataset({
        features: ['idx', 'count', 'hate_speech', 'offensive_language', 'neither', 'class', 'tweet', 'input_ids', 'attention_mask'],
        num_rows: 12392
    })
})
{'hate_speech': 1, 'offensive_language': 0, 'neither': 0, 'input_ids': [0, 13963, 787, 510, 3036, 611, 6106, 5449, 35, 38, 213, 7, 185, 5, 8875, 66, 8, 10, 385, 15921, 4245, 44831, 2538, 8268, 127, 16490, 11671, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [10]:
train_dataloader = DataLoader(tokenized_data['train'], batch_size=8)
test_dataloader = DataLoader(tokenized_data['test'], batch_size=8)

dataiter = iter(test_dataloader)
batch_data= next(dataiter)

print(batch_data.keys())

dict_keys(['hate_speech', 'offensive_language', 'neither', 'input_ids', 'attention_mask'])


In [11]:
from transformers import get_scheduler
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
# feel free to experiment with different num_warmup_steps
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=1, num_training_steps=num_training_steps
)

In [12]:
import evaluate
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

num_epochs = 2

for epoch in range(num_epochs):
    for batch in train_dataloader:
        print(batch)
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        logits = outputs.logits

        progress_bar.update(1)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/3098 [00:00<?, ?it/s]

{'hate_speech': tensor([1, 0, 0, 0, 0, 0, 1, 0]), 'offensive_language': tensor([0, 0, 1, 1, 1, 1, 0, 1]), 'neither': tensor([0, 1, 0, 0, 0, 0, 0, 0]), 'input_ids': tensor([[    0, 13963,   787,  ...,     1,     1,     1],
        [    0,   113,  1039,  ...,     1,     1,     1],
        [    0,  1185,  8265,  ...,     1,     1,     1],
        ...,
        [    0,   510,  5090,  ...,     1,     1,     1],
        [    0,  1039,   119,  ...,     1,     1,     1],
        [    0,   113,  1039,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


TypeError: forward() got an unexpected keyword argument 'hate_speech'